In [32]:
#Get Credentials

import os
import google_auth_oauthlib
from googleapiclient.discovery import build
from IPython.display import JSON
import pandas as pd

api_key = '{API_KEY}'


api_service_name = "youtube"
api_version = "v3"

def Youtube():
    # Get credentials and create an API client
    youtube = build(
    api_service_name, api_version, developerKey=api_key)
    return youtube

In [33]:
# Get general channel information

channel_ids = ["UCnItr-I_IeD9kv2zC2yLcFg","UCDWnJMuJXwnMG9Pm7ERoVYA"]

yt = Youtube()



request = yt.channels().list(
    part="snippet,contentDetails,statistics",
    id=','.join(channel_ids)
)
response = request.execute()

In [34]:
# Loop through items
def channel_stats(youtube,channel_ids):
    all_data = []
    for item in response['items']:
        data = {
            'channelname': item['snippet']['title'],
            'channelStartDate': item['snippet']['publishedAt'],
            'Country': item['snippet']['country'],
            'subscribers': item['statistics']['subscriberCount'],
            'views': item['statistics']['viewCount'],
            'TotalVidoes': item['statistics']['videoCount'],
            'PlayListID': item['contentDetails']['relatedPlaylists']['uploads']
        }
        all_data.append(data)
    return pd.DataFrame(all_data)

channel_stats_df = channel_stats(yt,channel_ids)

print(channel_stats_df)

   channelname      channelStartDate Country subscribers     views  \
0    Steffi M.  2016-02-22T10:38:59Z      CH         642     49177   
1  Lydia Dinga  2009-09-26T08:29:24Z      GB      229000  29194261   

  TotalVidoes                PlayListID  
0          23  UUDWnJMuJXwnMG9Pm7ERoVYA  
1         443  UUnItr-I_IeD9kv2zC2yLcFg  


In [53]:
## Get all the videos

playlistId=["UUnItr-I_IeD9kv2zC2yLcFg","UUDWnJMuJXwnMG9Pm7ERoVYA"]


def get_all_video_ids(youtube, playlistIds):
    all_video_ids = []  # This will store video IDs from all playlists

    for playlistId in playlistIds:  # Loop through each playlist ID
        video_ids = []  # Temporary list to store video IDs for the current playlist
        request = youtube.playlistItems().list(
            part="snippet,contentDetails",
            playlistId=playlistId,
            maxResults=50
        )
        response = request.execute()

        for item in response['items']:
            video_ids.append(item['contentDetails']['videoId'])

        next_page_token = response.get('nextPageToken')
        while next_page_token:
            request = youtube.playlistItems().list(
                part="snippet,contentDetails",
                playlistId=playlistId,
                maxResults=50,
                pageToken=next_page_token  # Use the next_page_token for subsequent requests
            )
            response = request.execute()
            for item in response['items']:
                video_ids.append(item['contentDetails']['videoId'])
            next_page_token = response.get('nextPageToken')

        all_video_ids.extend(video_ids)  # Add the video IDs from the current playlist to the master list

    return all_video_ids


In [54]:
video_ids = get_all_video_ids(Youtube(),playlistId)

In [55]:
len(video_ids)

466

In [68]:
# Check the details of the videos

def get_video_details(youtube,video_ids):
    
    all_video_info = []

    for i in range(0,len(video_ids),50):
        request = yt.videos().list(
            part="snippet,contentDetails,statistics",
            id=','.join(video_ids[i:i+50])
        )
        response = request.execute()
        
        for video in response['items']:
            stats_to_keep = {
                'snippet':['channelTitle','title','description','tags','publishedAt'],
                'statistics':['viewCount','likeCount','favouriteCount','commentCount'],
                'contentDetails':['duration','definition','caption']
            }
            video_info = {}
            video_info['video_id'] = video['id']
        
            for k in stats_to_keep.keys():
                for v in stats_to_keep[k]:
                    try:
                        video_info[v] = video[k][v]
                    except:
                        video_info[v] = None
            all_video_info.append(video_info)
    return pd.DataFrame(all_video_info)

In [69]:
video_df = get_video_details(Youtube(),video_ids)

In [70]:
video_df

,video_id,channelTitle,title,description,tags,publishedAt,viewCount,likeCount,favouriteCount,commentCount,duration,definition,caption
0,8I02WCiixvw,Lydia Dinga,WE BOUGHT A HOUSE!🏠🔑 | Victorian Fixer-upper ...,God did it for us! Join us on a tour of our ne...,"[lydia dinga, lydiadinga, vlogs, weekly vlog, ...",2024-04-07T18:38:06Z,27986,3664,None,464,PT9M59S,hd,false
1,lAlVj7ETAxA,Lydia Dinga,Honoured on Home Grounds 🇰🇪🥂♾️,,None,2024-03-31T14:32:52Z,10583,2388,None,100,PT45S,hd,false
2,qHpmxPaek6w,Lydia Dinga,"NAIROBI, OUTSIDE, RECKLESS ONE NIGHT: DAYS IN ...",-------------------------------\n\nFOLLOW ME ▹...,"[lydia dinga, lydiadinga, vlogs, day in my lif...",2024-03-24T19:37:51Z,40782,2643,None,316,PT30M46S,hd,false
3,_x0-3jlaCNI,Lydia Dinga,IT'S FINALLY TIME! WE'VE BEEN PLANNING THIS FO...,0:00 This is on my vision board \n05:22 Shoot ...,"[lydia dinga, lydiadinga, vlogs, day in my lif...",2024-03-17T13:00:06Z,35105,1898,None,125,PT36M29S,hd,false
4,LSesAX-Bbh4,Lydia Dinga,"THIS WAS A TERRIBLE IDEA, THE CHAOS OF MOVING ...",-------------------------------\n\nFOLLOW ME ▹...,"[lydia dinga, lydiadinga, vlogs, day in my lif...",2024-03-10T18:00:22Z,32688,2124,None,179,PT57M1S,hd,false
...,...,...,...,...,...,...,...,...,...,...,...,...,...
461,LgvJxtbZ4Mc,Steffi M.,Life in Switzerland as a Black | Is there Raci...,Hey there sweeties!\nI hope everybody is doing...,"[Racism, switzerland, zurich, zürich, black, g...",2020-07-11T11:50:44Z,18909,404,None,223,PT26M56S,hd,false
462,fl8OjQFqX5Y,Steffi M.,SLEEK LOW BUN / LOW PUFF ON SHORT 4C HAIR,Hey Loves\nI Hope you're all doing good. In th...,"[4c, 4c hair, sleek, low bun, low puff, hair t...",2020-05-23T14:34:05Z,307,32,None,15,PT11M35S,hd,false
463,wu-z-0Ps5iw,Steffi M.,EASY EYEBROW TUTORIAL | CLOSE UP | BEGINNER FR...,Hey Loves\nThis is a quick and easy tutorial o...,"[Eyebrow, tutorial, eyebrow tutorial, makeup, ...",2020-04-29T18:45:22Z,171,25,None,15,PT5M41S,hd,false
464,BJnirqVIbBY,Steffi M.,I FORGOT MY MAKEUP ROUTINE. FIRST MAKEUP VIDEO...,Hey Guys\nI hope everyone is doing fine especi...,"[Makeup routine, makeup, darkskin, darksinmake...",2020-04-16T19:07:02Z,235,37,None,22,PT11M55S,hd,false


In [71]:
def get_comments(yt,video_ids):
    all_comments = []

    for video_id in video_ids:
        request = yt.commentThreads().list(
            part="snippet,replies",
            videoId=video_id
        )
        response = request.execute()

        comments_in_video = [comment['snippet']['topLevelComment']['snippet']['textOriginal'] for comment in response['items']]
        comments_in_video_info = {'video_id':video_id,'comments':comments_in_video}

        all_comments.append(comments_in_video_info)
    return pd.DataFrame(all_comments)


In [72]:
comments_df = get_comments(Youtube(),video_ids)

In [73]:
comments_df

,video_id,comments
0,8I02WCiixvw,[EIIIIKK!!! I’m so excited to finally be shari...
1,lAlVj7ETAxA,[Ankara fashions are so beautiful and glamorou...
2,qHpmxPaek6w,[Love the short cut. You could rock a pixie cu...
3,_x0-3jlaCNI,[Sasa babes! I hope you guys enjoy the vlog be...
4,LSesAX-Bbh4,"[Yoh Lydia❤ I see you,, Rubbing hand sanitizer..."
...,...,...
461,LgvJxtbZ4Mc,[Thanks for making this informative video. I w...
462,fl8OjQFqX5Y,"[Love it!!! Thank you, 🌬🔥🔥🔥, I love this Nneom..."
463,wu-z-0Ps5iw,[Oh you did that sis!!!!!! Great tutorial. New...
464,BJnirqVIbBY,"[🌬🌹, beautiful!! 😍💕, 5:55 Bless You!\n\n\n\n\n..."


## Data Preproccessing

In [74]:
video_df.dtypes

video_id          object
channelTitle      object
title             object
description       object
tags              object
publishedAt       object
viewCount         object
likeCount         object
favouriteCount    object
commentCount      object
duration          object
definition        object
caption           object
dtype: object

In [75]:
numeric_cols = ["viewCount","likeCount","favouriteCount","commentCount"]
video_df[numeric_cols] = video_df[numeric_cols].apply(pd.to_numeric,errors = 'coerce',axis =1)

In [76]:
video_df.dtypes

video_id           object
channelTitle       object
title              object
description        object
tags               object
publishedAt        object
viewCount         float64
likeCount         float64
favouriteCount    float64
commentCount      float64
duration           object
definition         object
caption            object
dtype: object

In [80]:
video_df['publishedAt'] = pd.to_datetime(video_df['publishedAt'])

In [83]:
video_df['durationSecs'] = video_df['duration'].apply(lambda x: isodate.parse_duration(x))
video_df['durationSecs'] = video_df['durationSecs'].astype('timedelta64[s]')

In [84]:
video_df.dtypes

video_id                       object
channelTitle                   object
title                          object
description                    object
tags                           object
publishedAt       datetime64[ns, UTC]
viewCount                     float64
likeCount                     float64
favouriteCount                float64
commentCount                  float64
duration                       object
definition                     object
caption                        object
durationSecs           timedelta64[s]
dtype: object

In [85]:
video_df["tagCount"] = video_df['tags'].apply(lambda x: 0 if x is None else len(x))

In [ ]:
## Write to excel

In [87]:
video_df.to_csv('Video_Details.csv', index=False)

In [88]:
comments_df.to_csv('Comments_Details.csv', index=False)

In [89]:
channel_stats_df.to_csv('Channel_Details.csv', index=False)